In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import Decision_Tree as dt
import math

In [2]:
data=pd.read_csv('data3_19.csv')
test=pd.read_csv('test3_19.csv',header=None)

In [3]:
data.head()

,pclass,age,gender,survived
0,1st,adult,male,yes
1,1st,adult,male,yes
2,1st,adult,male,yes
3,1st,adult,male,yes
4,1st,adult,male,yes


In [4]:
test.head()

,0,1,2,3
0,1st,adult,male,yes
1,1st,adult,male,yes
2,1st,adult,male,yes
3,1st,adult,male,yes
4,1st,adult,male,yes


In [5]:
print(data.shape)
print(test.shape)

(2150, 4)
(66, 4)


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2150 entries, 0 to 2149
Data columns (total 4 columns):
pclass      2150 non-null object
age         2150 non-null object
gender      2150 non-null object
survived    2150 non-null object
dtypes: object(4)
memory usage: 67.3+ KB


In [7]:
data['pclass'].value_counts()

crew    885
3rd     699
1st     307
2nd     259
Name: pclass, dtype: int64

In [8]:
data['age'].value_counts()

adult    2046
child     104
Name: age, dtype: int64

In [9]:
data['gender'].value_counts()

male      1710
female     440
Name: gender, dtype: int64

In [56]:
training=data.to_numpy()
N=training.shape[0]
indices=[i for i in range(0,N)]
init_prob=1/N
weight=[init_prob for i in range(0,N)]
T=3
#features=np.array(['pclass','age','gender'])
features={}
for feature in data.columns.values[:3]:
    features[feature]=data[feature].value_counts().index.values
attribute={'pclass':0,'age':1,'gender':2,'survived':3}
Trees=[]
Classifier_Weight=[]
for i in range(0,T):
    sample_indices=np.random.choice(indices,N,p=weight)
    sample_data=training[[sample_indices]]
    tree=Tree(attribute)
    root=tree.build_tree(sample_data,features)
    #tree.print_tree(root)
    error=0
    misclassified=[]
    correctclassified=[]
    for j in range(0,N):
        if(sample_data[j,3]!=tree.Prediction(sample_data[j])):
            error+=weight[sample_indices[j]]
            misclassified.append(sample_indices[j])
        else:
            correctclassified.append(sample_indices[j])
    #print(list(sample_data[:,3]).count('yes'),"  ",list(sample_data[:,3]).count('no'))
    if(error==0): error=1e-4
    print(error)
    alpha=(1/2)*math.log(abs(1-error)/error)
    for j in misclassified:
        weight[j]=weight[j]*math.exp(alpha)
    for j in correctclassified:
        weight[j]=weight[j]*math.exp(-alpha)
    s=sum(weight)
    weight=[k/s for k in weight]
    
    Trees.append(tree)
    Classifier_Weight.append(alpha)
    

0.19953488372092854
0.5246534857537647
0.29150969715135766


/home/ayan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  app.launch_new_instance()


In [28]:
target={'yes':1,'no':-1}

def Predict(test):
    Predictions=[]
    s=0
    for instance in test:
        Classifier=0
        for i in range(len(Trees)):
            Classifier+=Classifier_Weight[i]*target[Trees[i].Prediction(instance)]
        if Classifier>=0:
            Predictions.append('yes')
        else: Predictions.append('no')
        
        if(Predictions[-1]==instance[3]): s+=1
    print(s/len(test))
        
    return Predictions

In [59]:
print("Training")
p_train=Predict(training)

print("Testing")
p_test=Predict(test.to_numpy())

Training
0.7869767441860465
Testing
0.5303030303030303


In [53]:
s=0
for i in range(len(training)):
    if training[i][3]==p_train[i]: 
        s+=1
print("Training: {}".format(s/len(training)))
s=0
t=test.to_numpy()
for i in range(len(t)):
    if t[i][3]==p_test[i]: 
        s+=1
print("Testing: {}".format(s/len(t)))

Training: 0.4753488372093023
Testing: 0.8181818181818182


In [10]:
def entropy(S):
    p1=list(S[:,3]).count('yes')/(S.shape[0])
    p0=list(S[:,3]).count('no')/(S.shape[0])
    if(p0==0): return -p1*math.log2(p1)
    if(p1==0): return -p0*math.log2(p0)
    return -p0*math.log2(p0)-p1*math.log2(p1)

def gain(S,A):
    """
       S-> data
       A-> splitting attribute
    """
    unique_values=np.unique(S[:,A])
    S_len=S.shape[0]
    entropy_Sv=0
    for v in unique_values:
        Sv=S[S[:,A]==v]
        entropy_Sv+=(Sv.shape[0]/S_len)*entropy(Sv)
    
    return entropy(S)-entropy_Sv

class node:
    
    def __init__(self,data,value=None,splitting_attr=None,branches=[],leaf=False,predict=None):
        self.value=value
        self.splitting_attr=splitting_attr
        self.branches=branches
        self.leaf=leaf
        if(leaf):
            self.predict=predict
class Tree:
    def __init__(self,attribute):
        self.root=None
        self.attribute=attribute
        
    def build_tree(self,training,features,max_tree_depth=4,value=None,l=0):
        
        p1=list(training[:,3]).count('yes')
        p0=list(training[:,3]).count('no')
        
        if(p1==0): 
            self.root=node(training,value,leaf=True,predict='no')
            return self.root
        if(p0==0): 
                self.root=node(training,value,leaf=True,predict='yes')
                return self.root
        
        if(len(features)==0 ):
            if(p0>p1): 
                self.root=node(training,value,leaf=True,predict='no')
                return self.root
            else: 
                self.root=node(training,value,leaf=True,predict='yes')
                return self.root
        
        max_gain=-1
        splitting_attr=''
        for i in features.keys():
            ig=gain(training,self.attribute[i])
            if(ig>max_gain):
                max_gain=ig
                splitting_attr=i
        branches=[]
       
        for v in features[splitting_attr]:
            if(training[training[:,self.attribute[splitting_attr]]==v].size==0): 
                    if(p0>p1): branches.append(node(training,value=v,leaf=True,predict='no'))
                    else: branches.append( node(training,value=v,leaf=True,predict='yes'))
            else:
                f={key:val for key,val in features.items() if key!=splitting_attr}
                branches.append(self.build_tree(training[training[:,self.attribute[splitting_attr]]==v],f,
                                      max_tree_depth,value=v,l=l+1) )
            
        root=node(training,value,splitting_attr,branches)
        self.root=root
        return root
  
    def Prediction(self,test):
        r=self.root
        while(r.leaf!=True):
            a=r.splitting_attr
            for node in r.branches:
                if(test[self.attribute[a]]==node.value):
                    r=node
                    break
        return r.predict
    def print_tree(self,root,ind='   ',l=0,splitting_attr=None):
        if(root.leaf==True):
            if(splitting_attr!=None):
                print(l*ind,splitting_attr," = ",root.value,": ",root.predict)
                return
            else: 
                print(l*ind,"survived = ",root.predict)
                return
        if(splitting_attr!=None): print(l*ind,splitting_attr," = ",root.value)
        
        for node in root.branches:
            self.print_tree(node,l=l+1,splitting_attr=root.splitting_attr)

In [ ]:
class Tree_stump:
    def __init__(self,attribute):
        self.root=None
        self.attribute=attribute
        
    def build_tree(self,training,features,max_tree_depth=4,value=None,l=0):
        for feature in features.key():
            for v in features[feature]:
                training[training[:,self.attribute[feature]]==v]
        
  
    def Prediction(self,test):
        r=self.root
        while(r.leaf!=True):
            a=r.splitting_attr
            for node in r.branches:
                if(test[self.attribute[a]]==node.value):
                    r=node
                    break
        return r.predict
    def print_tree(self,root,ind='   ',l=0,splitting_attr=None):
        if(root.leaf==True):
            if(splitting_attr!=None):
                print(l*ind,splitting_attr," = ",root.value,": ",root.predict)
                return
            else: 
                print(l*ind,"survived = ",root.predict)
                return
        if(splitting_attr!=None): print(l*ind,splitting_attr," = ",root.value)
        
        for node in root.branches:
            self.print_tree(node,l=l+1,splitting_attr=root.splitting_attr)

In [ ]:
N=training.shape[0]
indices=[i for i in range(0,N)]
init_prob=1/N
weight=[init_prob for i in range(0,N)]

T=20
features=np.array(['pclass','age','gender'])
attribute={'pclass':0,'age':1,'gender':2,'survived':3}
Trees=[]
Classifier_Weight=[]
for i in range(0,T):
    sample_indices=np.random.choice(indices,N,p=weight)
    sample_data=training[[sample_indices]]
    tree=Tree(attribute)
    root=tree.build_tree(sample_data,features)
    #tree.print_tree(root)
    error=0
    misclassified=[]
    correctclassified=[]
    for j in range(0,N):
        if(sample_data[j,3]!=tree.Prediction(sample_data[j])):
            error+=weight[sample_indices[j]]
            misclassified.append(sample_indices[j])
        else:
            correctclassified.append(sample_indices[j])
    print(error)
    if(error==0): error=1e-5
    alpha=(1/2)*math.log(abs(1-error)/error)
    for j in misclassified:
        weight[j]=weight[j]*math.exp(alpha)
    for j in correctclassified:
        weight[j]=weight[j]*math.exp(-alpha)
    s=sum(weight)
    weight=[k/s for k in weight]
    
    Trees.append(tree)
    Classifier_Weight.append(alpha)
    